<a href="https://www.kaggle.com/code/smaidebbarma/movie-rating-prediction?scriptVersionId=265931690" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['movielens', 'dataset']


In [2]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

**1.  Data acquisition of the movielens dataset**

In [3]:
#train data
train_cols = ['timestamp','user_id', 'item_id', 'rating' ]
df_train = pd.read_csv(
    '/kaggle/input/dataset/train.csv',
    sep=','
)
df_train.head()

,timestamp,user_id,item_id,rating
0,874965758,1,1,5
1,876893171,1,2,3
2,878542960,1,3,4
3,876893119,1,4,3
4,889751712,1,5,3


In [4]:
#users data
user_col = ['user_id','age', 'gender', 'occupation','zip_code' ]
df_user = pd.read_csv(
    '/kaggle/input/dataset/user.csv',
    sep=','
)
df_user.head()

,user_id,age,gender,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [5]:
#item data  
cols = ['movie_id','title','release_date','imdb_url','unknown','Action','Adventure','Animation',"Children's",'Comedy','Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western']
df_item = pd.read_csv("/kaggle/input/dataset/item.csv",
        sep=',',
        encoding='latin-1'
)
df_item.columns = df_item.columns.str.strip()
df_item.drop(columns=['unknown','release_date','imdb_url'], axis=1, inplace=True)
df_item.head()

,movie_id,title,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0.0
1,2,GoldenEye (1995),1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.0
2,3,Four Rooms (1995),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.0
3,4,Get Shorty (1995),1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0.0
4,5,Copycat (1995),0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0.0


In [6]:
#test data
test_cols = ['id','user_id', 'item_id']
df_test = pd.read_csv(
    '/kaggle/input/dataset/test.csv',
    sep=','
)
df_test.head()

,id,user_id,item_id
0,1,1,20
1,2,1,33
2,3,1,61
3,4,1,117
4,5,1,155


In [7]:
# train item id is actually movie id renamed it
df_train.rename(columns={'item_id':'movie_id'},inplace=True)

#train
df_merge=pd.merge(df_train,df_item,on='movie_id',how='left')
df_train_final=pd.merge(df_merge,df_user,on='user_id',how='left')

y_train = df_train_final['rating']

print(df_train_final.shape)
df_train_final.head()

(90570, 27)


,timestamp,user_id,movie_id,rating,title,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,age,gender,occupation,zip_code
0,874965758,1,1,5,Toy Story (1995),0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0.0,24,M,technician,85711
1,876893171,1,2,3,GoldenEye (1995),1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.0,24,M,technician,85711
2,878542960,1,3,4,Four Rooms (1995),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.0,24,M,technician,85711
3,876893119,1,4,3,Get Shorty (1995),1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0.0,24,M,technician,85711
4,889751712,1,5,3,Copycat (1995),0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0.0,24,M,technician,85711


In [8]:
#test
df_test.rename(columns={'item_id':'movie_id'},inplace=True)

df_merge=pd.merge(df_test,df_item,on='movie_id',how='left')
df_test_final=pd.merge(df_merge,df_user,on='user_id',how='left')
print(df_test_final.shape)
df_test_final.head()

(9430, 26)


,id,user_id,movie_id,title,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,age,gender,occupation,zip_code
0,1,1,20,Angels and Insects (1995),0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0.0,24,M,technician,85711
1,2,1,33,Desperado (1995),1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0.0,24,M,technician,85711
2,3,1,61,Three Colors: White (1994),0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.0,24,M,technician,85711
3,4,1,117,Rock; The (1996),1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.0,24,M,technician,85711
4,5,1,155,Dirty Dancing (1987),0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0.0,24,M,technician,85711


In [9]:
user_mean = (df_train_final.groupby('user_id', as_index=False)['rating'].mean().rename(columns={'rating': 'user_avg_rating'}))

movie_mean = (df_train_final.groupby('movie_id', as_index=False)['rating'].mean().rename(columns={'rating': 'movie_avg_rating'}))

global_mean = df_train_final['rating'].mean()

df_train_final = pd.merge(df_train_final, user_mean, on='user_id', how='left')
df_train_final = pd.merge(df_train_final, movie_mean, on='movie_id', how='left')

df_test_final = pd.merge(df_test_final, user_mean, on='user_id', how='left')
df_test_final = pd.merge(df_test_final, movie_mean, on='movie_id', how='left')

#global mean

df_train_final['global_mean'] = global_mean
df_test_final['global_mean'] = global_mean

In [10]:
df_test_final.fillna(global_mean, inplace=True)
df_train_final.fillna(global_mean, inplace=True)

In [11]:
df_test_final.isnull().sum()

id                  0
user_id             0
movie_id            0
title               0
Action              0
Adventure           0
Animation           0
Children's          0
Comedy              0
Crime               0
Documentary         0
Drama               0
Fantasy             0
Film-Noir           0
Horror              0
Musical             0
Mystery             0
Romance             0
Sci-Fi              0
Thriller            0
War                 0
Western             0
age                 0
gender              0
occupation          0
zip_code            0
user_avg_rating     0
movie_avg_rating    0
global_mean         0
dtype: int64

In [12]:
df_test_final.isnull().sum()

id                  0
user_id             0
movie_id            0
title               0
Action              0
Adventure           0
Animation           0
Children's          0
Comedy              0
Crime               0
Documentary         0
Drama               0
Fantasy             0
Film-Noir           0
Horror              0
Musical             0
Mystery             0
Romance             0
Sci-Fi              0
Thriller            0
War                 0
Western             0
age                 0
gender              0
occupation          0
zip_code            0
user_avg_rating     0
movie_avg_rating    0
global_mean         0
dtype: int64

# Feature engineering or encoding

In [13]:
#y_train_deviation = df_train_final['rating'] - df_train_final['user_avg_rating'] 

In [14]:
#Features Encoding
categorical_cols=['gender','occupation']

df_train_features = df_train_final.drop('rating',axis=1)

df_combined=pd.concat([df_train_features,df_test_final],ignore_index=True,sort=False)
df_encoded=pd.get_dummies(df_combined,columns=categorical_cols,drop_first=True)

df_encoded.head()

,timestamp,user_id,movie_id,title,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,age,zip_code,user_avg_rating,movie_avg_rating,global_mean,id,gender_M,occupation_artist,occupation_doctor,occupation_educator,occupation_engineer,occupation_entertainment,occupation_executive,occupation_healthcare,occupation_homemaker,occupation_lawyer,occupation_librarian,occupation_marketing,occupation_none,occupation_other,occupation_programmer,occupation_retired,occupation_salesman,occupation_scientist,occupation_student,occupation_technician,occupation_writer
0,874965758.0,1,1,Toy Story (1995),0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0.0,24,85711,3.603053,3.859694,3.523827,NaN,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,876893171.0,1,2,GoldenEye (1995),1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.0,24,85711,3.603053,3.198347,3.523827,NaN,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,878542960.0,1,3,Four Rooms (1995),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.0,24,85711,3.603053,3.058824,3.523827,NaN,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,876893119.0,1,4,Get Shorty (1995),1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0.0,24,85711,3.603053,3.545455,3.523827,NaN,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,889751712.0,1,5,Copycat (1995),0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0.0,24,85711,3.603053,3.291139,3.523827,NaN,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [15]:

train_size = len(df_train_final)
X_train = df_encoded.iloc[:train_size]
X_test = df_encoded.iloc[train_size:]

cols_to_drop = ['timestamp', 'zip_code', 'title', 'id', 'user_id', 'movie_id']
X_train = X_train.drop(cols_to_drop, axis=1, errors='ignore')
X_test = X_test.drop(cols_to_drop, axis=1, errors='ignore')

y_train=y_train
print("Done")

Done


In [16]:
X_train.head()

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,age,user_avg_rating,movie_avg_rating,global_mean,gender_M,occupation_artist,occupation_doctor,occupation_educator,occupation_engineer,occupation_entertainment,occupation_executive,occupation_healthcare,occupation_homemaker,occupation_lawyer,occupation_librarian,occupation_marketing,occupation_none,occupation_other,occupation_programmer,occupation_retired,occupation_salesman,occupation_scientist,occupation_student,occupation_technician,occupation_writer
0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0.0,24,3.603053,3.859694,3.523827,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.0,24,3.603053,3.198347,3.523827,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.0,24,3.603053,3.058824,3.523827,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0.0,24,3.603053,3.545455,3.523827,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0.0,24,3.603053,3.291139,3.523827,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [17]:
X_test.head()

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,age,user_avg_rating,movie_avg_rating,global_mean,gender_M,occupation_artist,occupation_doctor,occupation_educator,occupation_engineer,occupation_entertainment,occupation_executive,occupation_healthcare,occupation_homemaker,occupation_lawyer,occupation_librarian,occupation_marketing,occupation_none,occupation_other,occupation_programmer,occupation_retired,occupation_salesman,occupation_scientist,occupation_student,occupation_technician,occupation_writer
90570,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0.0,24,3.603053,3.344262,3.523827,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
90571,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0.0,24,3.603053,3.460674,3.523827,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
90572,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.0,24,3.603053,3.857143,3.523827,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
90573,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.0,24,3.603053,3.690625,3.523827,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
90574,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0.0,24,3.603053,3.093750,3.523827,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [18]:
X_test.isnull().sum()

Action                      0
Adventure                   0
Animation                   0
Children's                  0
Comedy                      0
Crime                       0
Documentary                 0
Drama                       0
Fantasy                     0
Film-Noir                   0
Horror                      0
Musical                     0
Mystery                     0
Romance                     0
Sci-Fi                      0
Thriller                    0
War                         0
Western                     0
age                         0
user_avg_rating             0
movie_avg_rating            0
global_mean                 0
gender_M                    0
occupation_artist           0
occupation_doctor           0
occupation_educator         0
occupation_engineer         0
occupation_entertainment    0
occupation_executive        0
occupation_healthcare       0
occupation_homemaker        0
occupation_lawyer           0
occupation_librarian        0
occupation

# Feature Scaling

# Modelling

In [19]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error
#import lightgbm as lgb

In [20]:
X_train_final, X_val, y_train_final, y_val = train_test_split(
    X_train, y_train, test_size=0.10, random_state=1
)

In [21]:
#optimized
xgb_model_final  = xgb.XGBRegressor(
     objective='reg:squarederror', 
    n_estimators=7000, 
    learning_rate=0.006, #0.01
    max_depth=6,#9
    subsample=0.7 ,#0.8
    min_child_weight=7,   
    colsample_bytree=0.7,#0.8
    reg_alpha=1.2 ,#0.8
    reg_lambda=2.5 ,#1.5
    random_state=1,
    n_jobs=-1
)

#early stop fit
xgb_model_final.fit(X_train_final, y_train_final,
                    eval_set=[(X_val, y_val)], 
                    eval_metric="rmse", 
                    early_stopping_rounds=100, verbose=False)

print("Done")

/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


Done


In [22]:
#predictions
#test_pred = xgb_model_final.predict(X_test_scaled).clip(1.0,5.0).round().astype(int)
predictions = xgb_model_final.predict(X_test)

# Final Rating = predictions_deviation + user_avg_rating
#final_ratings = predictions_deviation + df_test_final['user_avg_rating']

#make the range from 1 to 5
predictions=predictions .clip(1.0,5.0)
#val_predictions = val_predictions.clip(1.0, 5.0)

predictions_int=predictions.round().astype(int)
#mse_score = mean_squared_error(y_val, val_predictions)
#rmse_score = np.sqrt(mse_score)

#print(f"Final Model Local Validation MSE: {mse_score:.5f}")
#print(f"Final Model Local Validation RMSE: {rmse_score:.5f}")

df_submissions=pd.DataFrame({
    'timestamp':df_test_final['id'],
    'rating':predictions_int
})

df_submissions.to_csv('submission.csv',index=False)
print("submission file created as submission.csv")
print(df_submissions.head())


submission file created as submission.csv
   timestamp  rating
0          1       3
1          2       4
2          3       4
3          4       4
4          5       3


**2.  Perform the Exploratory Data Analysis (EDA) for the users dataset**